In [29]:
#1차 데이터 전처리 특성중요도 0.01미만 54개 제거
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# 데이터 불러오기
data = pd.read_csv('./train_data.csv')

# 특성과 타겟 변수 분리
data_input = data.drop(columns=['Bankrupt?'])
data_target = data['Bankrupt?']

# 데이터 분리
train_input, test_input, train_target, test_target = train_test_split(
    data_input, data_target, random_state=42)

# 랜덤 포레스트 모델 학습
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(train_input, train_target)

# 특성 중요도 확인
importances = rf_model.feature_importances_
features = data_input.columns
feature_importances = pd.DataFrame({'Feature': features, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

# 중요도가 낮은 특성 제거 (중요도 0.01 미만인 특성 제거)
important_features = feature_importances[feature_importances['Importance'] > 0.01]['Feature']
data_input_reduced = data_input[important_features]

# 중요한 특성만 포함한 데이터프레임에 타겟 변수 'Bankrupt?' 컬럼 추가
data_reduced_with_target = data_input_reduced.copy()
data_reduced_with_target['Bankrupt?'] = data_target


data_reduced_with_target.to_csv('./train_data_reduced.csv', index=False)



In [31]:
# 제거할 특성 불러오기
low_importance_features = pd.read_csv('./low_importance_features.csv')['Feature'].tolist()

# 새로운 데이터 불러오기
test_data = pd.read_csv('./test_data.csv')

# 제거할 특성을 데이터에서 제거
test_data_reduced = test_data.drop(columns=low_importance_features)

# 중요한 특성만 포함된 새로운 데이터 저장
file_path_test_reduced = './test_data_reduced.csv'
test_data_reduced.to_csv(file_path_test_reduced, index=False)

file_path_test_reduced

'./test_data_reduced.csv'

In [32]:
#특성중요도 제거 전 후 비교(최종) 
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# 데이터 불러오기
data = pd.read_csv('./train_data.csv')

# 특성과 타겟 변수 분리
data_input = data.drop(columns=['Bankrupt?'])
data_target = data['Bankrupt?']

# 데이터 분리
train_input, test_input, train_target, test_target = train_test_split(
    data_input, data_target, random_state=42)

# 1. 특성을 제거하기 전 모델 성능 평가
# 랜덤 포레스트 모델 학습
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(train_input, train_target)

# 예측
train_predictions = rf_model.predict(train_input)
test_predictions = rf_model.predict(test_input)

# 모델 평가 - 정확도
train_accuracy = accuracy_score(train_target, train_predictions)
test_accuracy = accuracy_score(test_target, test_predictions)

# 모델 평가 - MSE, MAE
train_mse = mean_squared_error(train_target, train_predictions)
test_mse = mean_squared_error(test_target, test_predictions)
train_mae = mean_absolute_error(train_target, train_predictions)
test_mae = mean_absolute_error(test_target, test_predictions)

# 교차 검증 (Cross-Validation)
cv_scores = cross_val_score(rf_model, data_input, data_target, cv=5)
cv_accuracy = np.mean(cv_scores)

# 결과 출력
print("특성 제거 전 모델 성능:")
print(f"훈련 세트 정확도: {train_accuracy}")
print(f"테스트 세트 정확도: {test_accuracy}")
print(f"훈련 세트 MSE: {train_mse}")
print(f"테스트 세트 MSE: {test_mse}")
print(f"훈련 세트 MAE: {train_mae}")
print(f"테스트 세트 MAE: {test_mae}")
print("교차 검증 평균 정확도: ", cv_accuracy)

# 2. 특성을 제거한 후 모델 성능 평가
# 특성 중요도 확인 및 중요도가 0.01 이하인 특성 제거
importances = rf_model.feature_importances_
features = data_input.columns
feature_importances = pd.DataFrame({'Feature': features, 'Importance': importances})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
important_features = feature_importances[feature_importances['Importance'] > 0.01]['Feature']
data_input_reduced = data[important_features]

# 데이터 분리 (중요도가 낮은 특성 제거 후)
train_input_reduced, test_input_reduced, train_target_reduced, test_target_reduced = train_test_split(
    data_input_reduced, data_target, random_state=42)

# 랜덤 포레스트 모델 재학습
rf_model_reduced = RandomForestClassifier(random_state=42)
rf_model_reduced.fit(train_input_reduced, train_target_reduced)

# 예측
train_predictions_reduced = rf_model_reduced.predict(train_input_reduced)
test_predictions_reduced = rf_model_reduced.predict(test_input_reduced)

# 모델 평가 - 정확도
train_accuracy_reduced = accuracy_score(train_target_reduced, train_predictions_reduced)
test_accuracy_reduced = accuracy_score(test_target_reduced, test_predictions_reduced)

# 모델 평가 - MSE, MAE
train_mse_reduced = mean_squared_error(train_target_reduced, train_predictions_reduced)
test_mse_reduced = mean_squared_error(test_target_reduced, test_predictions_reduced)
train_mae_reduced = mean_absolute_error(train_target_reduced, train_predictions_reduced)
test_mae_reduced = mean_absolute_error(test_target_reduced, test_predictions_reduced)

# 교차 검증 (Cross-Validation)
cv_scores_reduced = cross_val_score(rf_model_reduced, data_input_reduced, data_target, cv=5)
cv_accuracy_reduced = np.mean(cv_scores_reduced)

# 결과 출력
print("특성 제거 후 모델 성능:")
print(f"훈련 세트 정확도: {train_accuracy_reduced}")
print(f"테스트 세트 정확도: {test_accuracy_reduced}")
print(f"훈련 세트 MSE: {train_mse_reduced}")
print(f"테스트 세트 MSE: {test_mse_reduced}")
print(f"훈련 세트 MAE: {train_mae_reduced}")
print(f"테스트 세트 MAE: {test_mae_reduced}")
print("교차 검증 평균 정확도: ", cv_accuracy_reduced)


특성 제거 전 모델 성능:
훈련 세트 정확도: 1.0
테스트 세트 정확도: 0.9699413489736071
훈련 세트 MSE: 0.0
테스트 세트 MSE: 0.030058651026392963
훈련 세트 MAE: 0.0
테스트 세트 MAE: 0.030058651026392963
교차 검증 평균 정확도:  0.9622364802933088
특성 제거 후 모델 성능:
훈련 세트 정확도: 1.0
테스트 세트 정확도: 0.9699413489736071
훈련 세트 MSE: 0.0
테스트 세트 MSE: 0.030058651026392963
훈련 세트 MAE: 0.0
테스트 세트 MAE: 0.030058651026392963
교차 검증 평균 정확도:  0.9644362969752521
